<a href="https://colab.research.google.com/github/Pablo1206/Ciencia-de-Datos-Python/blob/main/Cod_PRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ventas Diarias

In [ ]:
import pandas as pd
import numpy as np
import warnings
import os
import shutil
from datetime import datetime, timedelta
import win32com.client as win32
import time


#Definir Semana
ANO_ARCHIVO = 2024
N_Mes = "Jun"
Mes = "6"
Mescon0 = fr"0{Mes}"

### Definir DataFrame y valores de ANIO ###
df = pd.DataFrame()
Valores = [2024,"PPTO"]

### Eliminar Mensajes de Error ###
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl.worksheet._reader")


### Cargar Equivalencias ###

Rutas = "C:/Users/PabloMenendez/Reportería/Ventas Diarias/Anexos/"
Ruta_Semana = fr"{Rutas}PeriodosySem2023.xlsx"
Ruta_Pais = fr"{Rutas}Pais.xlsx"
Ruta_Cambio = fr"{Rutas}Cambio.xlsx"

Archivo_Semana = pd.read_excel(Ruta_Semana) # Periodo y semana de cada Fecha
Archivo_Pais = pd.read_excel(Ruta_Pais) # País de cada Marca
Archivo_Cambio = pd.read_excel(Ruta_Cambio) # Tipo de cambio por País

### Definir las rutas de Acceso de cada Archivo ###
# Obtener la ruta de los de los Archivos de Ventas Diarias
Ruta_BI = "C:\\Users\\PabloMenendez\\Franquicias Americanas SA de CV\\PRA-BI - PRA-BI\\"
carpeta_cwsv = fr"{Ruta_BI}5. CW CAM\\1. CW SV\\1. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_phgt = fr"{Ruta_BI}3. PH GT\\1. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_phsv = fr"{Ruta_BI}2. PH SV\\1. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_wdsv = fr"{Ruta_BI}7. WD SV\\1. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_stb = fr"{Ruta_BI}6. STB CAM\\0. STB CAM\\1. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_kfcsv = fr"{Ruta_BI}4. KFC CAM\\1. KFC SV\\0. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_kfcgt = fr"{Ruta_BI}4. KFC CAM\\2. KFC GT\\0. Ventas Diarias\\{ANO_ARCHIVO}\\"

# Crear un diccionario de rutas con las rutas relativas a la carpeta "Ventas Diarias"
Rutas = {fr"{carpeta_cwsv}{Mescon0}. Ventas Diarias CW SV {N_Mes} {ANO_ARCHIVO}.xlsx": ["Base DOMICILIO", "CW SV",3],
         fr"{carpeta_phgt}{Mes}. Ventas Diarias PH GT {N_Mes} {ANO_ARCHIVO}.xlsx": ["Pivot Domicilio","PH GT",1],
         fr"{carpeta_phsv}{Mescon0}. Ventas Diarias PH SV {N_Mes} {ANO_ARCHIVO}.xlsx": ["Base DOMICILIO","PH SV",3],
         fr"{carpeta_wdsv}{Mescon0}. Ventas Diarias WD SV {N_Mes} {ANO_ARCHIVO}.xlsx": ["Base DOMICILIO","WD SV",3],
         fr"{carpeta_stb}{Mescon0}. Ventas Diarias STB CAM {N_Mes} {ANO_ARCHIVO}.xlsx": ["Base DOMICILIO","STB CAM",3],
         fr"{carpeta_kfcsv}{Mescon0}. Ventas Diarias KFC SV {N_Mes} {ANO_ARCHIVO}.xlsx": ["Base DOMICILIO","KFC SV",3],
         fr"{carpeta_kfcgt}{Mes}. Ventas Diarias KFC GT {N_Mes} {ANO_ARCHIVO}.xlsx": ["Base DOMICILIO","KFC GT",3]
}


#Realizar la importación de datos para obtener la venta y el PPTO de cada marca
for ANIO in Valores:

    for marca, pivot in Rutas.items():
        #obtener el dataframe en base a las especificaciones de cada pivot

        marca = pd.read_excel(marca, sheet_name=pivot[0],skiprows=pivot[2])

        #filtrar por año
        marca = marca[marca['ANIO'] == ANIO]
        marca = marca.fillna('-')

        #Obtener mensaje de cuál es la fecha máxima del df
        marca['FECHA_DOCUMENTO'] = pd.to_datetime(marca['FECHA_DOCUMENTO'])
        fecha_maxima = marca['FECHA_DOCUMENTO'].max()
        print(pivot[1],ANIO,"Fecha máxima:", fecha_maxima)


        #Crear un groupby para evitar cualquier columna extra
        marca = marca.groupby(['Marca Pais','FECHA_DOCUMENTO', 'CANAL','OCASION',
                            'TIENDA','GerenteArea', 'CONCEPTO', 'TipoTienda','SUB CANAL', 'PLATAFORMA']
                            ).agg({'Suma de TRAFICO': 'sum', 'Suma de VENTA': 'sum'}).reset_index()


        #Agregar equivalencias
        marca = marca.merge(Archivo_Semana, on=['FECHA_DOCUMENTO'], how='left')
        marca = marca.merge(Archivo_Pais, on=['Marca Pais'], how='left')
        marca = marca.merge(Archivo_Cambio, on=['País','MES',"ANIO"], how='left')
        marca = marca.fillna('')

        #Cambiar el valor de ANIO PRA
        marca['ANIO PRA'] = marca['ANIO PRA'].astype(str)
        if ANIO == 'PPTO':
            marca['ANIO PRA'] = ANIO


        #Crear columnas de venta a cambio fijo y constante
        marca['$-Fijo']= marca['Suma de VENTA']/marca['$-Fijo']
        marca['$-Constante']= marca['Suma de VENTA']/marca['$-Constante']

        #Unir el df de la marca al consolidado
        df= pd.concat([df, marca], ignore_index=True)



    #Descargar el resultado en la carpeta de venta y PPTO
    ruta_salida1 = fr"C:/Users/PabloMenendez/Reportería/Ventas Diarias/BASE Ventas Diarias Excel/Ventas-{ANIO}.xlsx"
    df.to_excel(ruta_salida1, index=False)

    #Descargar el resultado en la carpeta sin PPTO y crear un df para el anexo de gerente
    if ANIO == 2024:
        ruta_salida2 = fr"C:/Users/PabloMenendez/Reportería/Ventas Diarias/Base VTA/Ventas-{ANIO}.xlsx"
        df.to_excel(ruta_salida2, index=False)

    #Reiniciar el df consolidado
    df = pd.DataFrame()
print("Proceso Finalizado")



Actualizar Archivos

In [ ]:

## Actualizar archivo de ventas diarias de domicilio
xlapp = win32.DispatchEx("Excel.Application")
xlapp.DisplayAlerts = False
xlapp.Visible = True
wb = xlapp.Workbooks.Open("C:/Users/PabloMenendez/Reportería/Ventas Diarias/Resultados Delivery Premium 2024.xlsx")
wb.RefreshAll()
time.sleep(15)
wb.Save()
wb.Close()
xlapp.Quit()
print("Archivos Actualizados")

## Copair archivo actualizado en carpetas compartidas
ruta_origen = "C:/Users/PabloMenendez/Reportería/Ventas Diarias/Resultados Delivery Premium 2024.xlsx"
ruta_Drive_Propio = "C:/Users/PabloMenendez/OneDrive - Franquicias Americanas SA de CV/Delivery Premium 2024/Ventas Diarias Delivery/Resultados Delivery Premium 2024.xlsx"
ruta_Drive_WF = "C:/Users/PabloMenendez/OneDrive - Franquicias Americanas SA de CV/WFM REPORTING/Documentos_BI/Resultados Delivery Premium/Resultados Delivery Premium 2024.xlsx"
# Copiar el archivo
shutil.copy(ruta_origen, ruta_Drive_Propio)
shutil.copy(ruta_origen, ruta_Drive_WF)

print("Archivo copiado con éxito.")

Supervisores

In [ ]:
ruta_Supervisor = "Y:\Documentos_BI\Base de datos Venta sugerida\Anexos\Base Roster para Query.xlsx"
Supervisores = pd.read_excel(ruta_Supervisor)
Supervisores = pd.melt(Supervisores,
                    id_vars=['Fecha', 'Nombre completo', 'Codigo de Empleado', 'Supervisor'],
                    value_vars=['Operador1', 'Operador2', 'Operador3'],
                    var_name='Borrar',
                    value_name='OPERADOR')
Supervisores= Supervisores.drop(columns=['Borrar'])
Supervisores= Supervisores.drop_duplicates()
Nombre_Columnas = ["FECHA","NOMBRE","CODIGO","SUPERVISOR","OPERADOR"]
Supervisores.columns = Nombre_Columnas

Archivos V Sugerida

In [ ]:
print("Comenzando la actualización")
#Definir Df consolidado
df_Vsug = pd.DataFrame()

#Obtener el número de mes actual
fecha = f_fin.strip("'")
fecha = datetime.strptime(fecha, '%Y-%m-%d')
mes = fecha.month

#Rutas de archivos de SQL
Ruta_SQL = "C:/Users/PabloMenendez/Reportería/Base de datos V_Sug/Consultas/"
Consultas = {
 fr"{Ruta_SQL}Base_Vsug_CW_SV.sql":["CW SV","172.16.100.207","ccrdb"]
,fr"{Ruta_SQL}Base_Vsug_WD_SV.sql":["WD SV","172.16.100.207","ccrdb"]
,fr"{Ruta_SQL}Base_Vsug_KFC_SV.sql":["KFC SV","172.16.100.207","ccrdb"]
,fr"{Ruta_SQL}Base_Vsug_PH_SV.sql":["PH SV","172.16.100.207","ccrdb"]
,fr"{Ruta_SQL}Base_Vsug_PH_GT.sql":["PH GT","172.17.76.215","ccgtdb"]
,fr"{Ruta_SQL}Base_Vsug_KFC_GT.sql":["KFC GT","172.17.76.215","ccgtdb"]
            }


#Base de datos
for marca, atributo in Consultas.items():

    #Abrir archivos de SQL
    BASE = marca
    with open(BASE, 'r') as sql_script:
        sentencia = sql_script.read()

    #Agregar fechas elegidas al archivo
    script_final = sentencia.replace(':F_INI', f_ini)
    script_final = script_final.replace(':F_FIN', f_fin)

    #Crear conexión a la base de datos
    p_dsn = ora.makedsn(atributo[1], 1521, sid = atributo[2])
    con = ora.connect(user = "BI_ANALISTA1", password = "PREMIUM2022#", dsn = p_dsn, encoding = "UTF-8")
    cursor = con.cursor()
    rs = cursor.execute(script_final)

    #Concatenar filas
    Base = []
    for fila in rs:
        Base.append(list(fila))
    con.close()
    print(f'Consulta {atributo[0]} Importada')

    #Crear nombres de Columnas
    df = pd.DataFrame(Base)
    df.columns = ['MARCA','FECHA','OPERADOR', 'PROGRAMA', 'CANT', 'VT', 'COM','Universo','CATEGORIA' ]
    df['MARCA']= str(atributo[0])

    #Obtner mes y anio
    df['FECHA'] = pd.to_datetime(df['FECHA'])
    df['FECHA'].dt.strftime('%d-%m-%Y')
    df['MES'] = df['FECHA'].dt.month
    df['ANO'] = df['FECHA'].dt.year

    #Concatenar archivo
    df_Vsug= pd.concat([df_Vsug,df], ignore_index=True)


#Definir Nombre del mes
NoMeses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
Carpeta = NoMeses[mes-1]

#Obtener Rutas de Equivalencias
ruta_metas = f"Y:/Documentos_BI/Base de datos Venta sugerida/Anexos/{Carpeta}/Metas.xlsx"
ruta_precios = f"Y:/Documentos_BI/Base de datos Venta sugerida/Anexos/{Carpeta}/Precios.xlsx"
#ruta_supervisores = f"Y:/Documentos_BI/Base de datos Venta sugerida/Anexos/{Carpeta}/Supervisores.xlsx"
ruta_Semanas = "Y:\Documentos_BI\Base de datos Venta sugerida\Anexos\PeriodosySem2023.xlsx"


#Agregar meta
Metas = pd.read_excel(ruta_metas)
df_Vsug = df_Vsug.merge(Metas, on=['MARCA','CATEGORIA'], how='left')
df_Vsug['Meta'] = df_Vsug['Meta'] * df_Vsug['Universo']

#Agregar Precios de productos
Precios = pd.read_excel(ruta_precios)
df_Vsug = df_Vsug.merge(Precios, on=['MARCA','PROGRAMA'], how='left')

def multiplicacion_selectiva(row):
    if row['MARCA'] not in ['PH GT', 'KFC GT']:
        return row['Monto'] * row['CANT']
    else:
        return row['Monto'] * row['VT']

df_Vsug['Monto'] = df_Vsug.apply(multiplicacion_selectiva, axis=1)


#Agregar Semanas
Semanas = pd.read_excel(ruta_Semanas)
df_Vsug = df_Vsug.merge(Semanas, on=['FECHA'], how='left')

#agregar supervisores y nombres
#Supervisores = pd.read_excel(ruta_supervisores)
df_Vsug = df_Vsug.merge(Supervisores, on=['FECHA','OPERADOR'], how='left')

#reemplazar Nan
valor_reemplazo = "-"
df_Vsug = df_Vsug.fillna(valor_reemplazo)


# Reemplazar NaN en la columna 'SUPERVISOR' con 'SIN SUPERVISOR'
df_Vsug['SUPERVISOR'] = df_Vsug['SUPERVISOR'].fillna('-')
df_Vsug['CODIGO'] = df_Vsug['CODIGO'].fillna('-')

# Reemplazar NaN en la columna 'NOMBRE' con 'SIN NOMBRE'
df_Vsug['NOMBRE'] = df_Vsug['NOMBRE'].fillna('-')


#Exportar el consolidado

df_Vsug.to_excel(fr'C:/Users/PabloMenendez/Reportería/Base de datos V_Sug/BASES/Base_VSug_{Carpeta}.xlsx', index=False)

df_Vsug.to_excel(fr'Y:/Documentos_BI/Base de datos Venta sugerida/BASES/Base_VSug_{Carpeta}.xlsx', index=False)

print("Consolidado Exportado Correctamente")




Cumplimiento

In [ ]:
#Obtener el número de mes actual
fecha = f_fin.strip("'")
fecha = datetime.strptime(fecha, '%Y-%m-%d')
mes = fecha.month
anio = fecha.year

#Definir Nombre del mes
NoMeses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
Carpeta = NoMeses[mes-1]

df_Fecha = pd.DataFrame()
#Consulta separada

Cont = 0
dfs = {}
Consultas = {
 "CWSV_Cumplimiento.sql":["CW SV","172.16.100.207","ccrdb"]
,"WDSV_Cumplimiento.sql":["WD SV","172.16.100.207","ccrdb"]
,"KFCSV_Cumplimiento.sql":["KFC SV","172.16.100.207","ccrdb"]
,"PHSV_Cumplimiento.sql":["PH SV","172.16.100.207","ccrdb"]
,"PHGT_Cumplimiento.sql":["PH GT","172.17.76.215","ccgtdb"]
,"KFCGT_Cumplimiento.sql":["KFC GT","172.17.76.215","ccgtdb"]
            }

df_Vsug = pd.DataFrame()
#Base de datos
for marca, atributo in Consultas.items():
    BASE = fr'C:\Users\PabloMenendez\Reportería\Base de datos V_Sug\Consultas\{marca}'
    with open(BASE, 'r') as sql_script:
        sentencia = sql_script.read()

    script_final = sentencia.replace(':F_INI', f_ini)
    script_final = script_final.replace(':F_FIN', f_fin)
    #Crear conexión a la base de datos
    p_dsn = ora.makedsn(atributo[1], 1521, sid = atributo[2])
    con = ora.connect(user = "BI_ANALISTA1", password = "PREMIUM2022#", dsn = p_dsn, encoding = "UTF-8")
    cursor = con.cursor()
    rs = cursor.execute(script_final)
    #Concatenar filas
    Base = []
    for fila in rs:
        Base.append(list(fila))
    con.close()
    print(f'Consulta {marca} importada')
    #transformaciones
    df = pd.DataFrame(Base)
    df.columns = ['FECHA','MARCA','OPERADOR','ORDENES','ORDENES_VSUG' ]
    df['MARCA']= str(atributo[0])

 # Almacenar el cada marca en el consolidado
    df_Fecha= pd.concat([df_Fecha, df], ignore_index=True)


#Agregar Nombres de Supervisores
ruta_supervisores = f"Y:/Documentos_BI/Base de datos Venta sugerida/Anexos/{Carpeta}/Supervisores.xlsx"
Supervisores = pd.read_excel(ruta_supervisores)
df_Fecha  = df_Fecha.merge(Supervisores, on=['OPERADOR'], how='left')

#Agregar Semanas
ruta_Semanas = "Y:\Documentos_BI\Base de datos Venta sugerida\Anexos\PeriodosySem2023.xlsx"
Semanas = pd.read_excel(ruta_Semanas)
df_Fecha = df_Fecha.merge(Semanas, on=['FECHA'], how='left')


#Eliminar Columnas
columnas_a_eliminar = ['TARDES', 'AUSENCIAS', 'VACACION']
df_Fecha = df_Fecha.drop(columnas_a_eliminar, axis=1)


# Reemplazar NaN en la columna 'SUPERVISOR' con 'SIN SUPERVISOR'
df_Fecha['SUPERVISOR'] = df_Fecha['SUPERVISOR'].fillna('-')
df_Fecha['CODIGO'] = df_Fecha['CODIGO'].fillna('-')

# Reemplazar NaN en la columna 'NOMBRE' con 'SIN NOMBRE'
df_Fecha['NOMBRE'] = df_Fecha['NOMBRE'].fillna('-')

#Exportar Df

df_Fecha.to_excel(fr'C:/Users/PabloMenendez/Reportería/Base de datos V_Sug/BASE_Cumplimineto/Base_VSug_Cump_{anio}_{Carpeta}.xlsx', index=False)

df_Fecha.to_excel(fr"Y:/Documentos_BI/Base de datos Venta sugerida/BASE_Cumplimineto/Base_VSug_Cump_{anio}_{Carpeta}.xlsx", index=False)


print(f'Exportado completo')

Menus mas vendidos

In [ ]:
import pandas as pd
import cx_Oracle as ora
from datetime import datetime, timedelta
import numpy as np
import win32com.client as win32


#Unicamente debes cambiar la fecha
#f_ini = f"'2024-{mes}-01'"
#f_fin = f"'{fecha}'"

#f_ini = f"'2024-02-01'"
#f_fin = f"'2024-02-29'"
fecha = f_fin.strip("'")
fecha = datetime.strptime(fecha, '%Y-%m-%d')
mes = fecha.month

Consultas = {
 "Menus_SV.sql":["172.16.100.207","ccrdb"]
,"Menus_GT.sql":["172.17.76.215","ccgtdb"]
            }

df_Menus = pd.DataFrame()

for marca, atributo in Consultas.items():

    BASE = fr'C:\Users\PabloMenendez\Reportería\Menus más Vendidos-old\Consultas\{marca}'
    with open(BASE, 'r') as sql_script:
        sentencia = sql_script.read()

    script_final = sentencia.replace(':F_INI', f_ini)
    script_final = script_final.replace(':F_FIN', f_fin)
    #Crear conexión a la base de datos
    p_dsn = ora.makedsn(atributo[0], 1521, sid = atributo[1])
    con = ora.connect(user = "BI_ANALISTA1", password = "PREMIUM2022#", dsn = p_dsn, encoding = "UTF-8")
    cursor = con.cursor()
    rs = cursor.execute(script_final)

    #Concatenar filas
    Base = []
    for fila in rs:
        Base.append(list(fila))
    con.close()
    print(f'Consulta {marca} Importada')
    #tranbsformaciones
    df = pd.DataFrame(Base)
    df.columns = ['MARCA','FECHA','TIENDA','ORIGEN','CODIGO','MENU','CANTIDAD','VENTA','VENTA SIN IVA']

    #MARCA REAL
    cond = [df["MARCA"] == "WDYSAL",df["MARCA"] == "KFCSAL",df["MARCA"] == "KFCGT"
           ,df["MARCA"] == "PHES",df["MARCA"] == "PHGUA",
           df["MARCA"] == "CWSAL"]
    value =["WD SV","KFC SV","KFC GT","PH SV","PH GT","CW SV"]

    df["MARCA"] = np.select(cond,value,default=0)


    df_Menus= pd.concat([df_Menus,df], ignore_index=True)




ruta_semanas = fr'C:\Users\PabloMenendez\Reportería\Menus más Vendidos-old\Consultas\PeriodosySem2023.xlsx'
SEMANAS = pd.read_excel(ruta_semanas)
df_Menus = df_Menus.merge(SEMANAS, on=['FECHA'], how='left')

ruta_Gerente = fr"C:\Users\PabloMenendez\Reportería\Menus más Vendidos-old\Consultas\Gerente.xlsx"
Gerente = pd.read_excel(ruta_Gerente)
df_Menus = df_Menus.merge(Gerente, on=['TIENDA','MARCA'], how='left')

#Guardar información del mes
NoMeses = [
    "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
]

Carpeta = NoMeses[mes-1]
df_Menus.to_excel(fr'C:\Users\PabloMenendez\Reportería\Menus más Vendidos-old\BASE/menus_{Carpeta}.xlsx', index=False)

Actualizar y pegar archivos

In [ ]:
#Actualizar archivo de Respaldo
xlapp = win32.DispatchEx("Excel.Application")
xlapp.DisplayAlerts = False
xlapp.Visible = True
wb = xlapp.Workbooks.Open(f"C:/Users/PabloMenendez/Reportería/Menus más Vendidos-old/Menús más vendidos PRA.xlsx")
wb.RefreshAll()
time.sleep(10)
wb.Save()
wb.Close()
xlapp.Quit()


ruta_origen = "C:/Users/PabloMenendez/Reportería/Menus más Vendidos-old/Menús más vendidos PRA.xlsx"

ruta_Drive_WF = "C:/Users/PabloMenendez/OneDrive - Franquicias Americanas SA de CV/WFM REPORTING/Documentos_BI/Menús Más Vendidos/Menús más vendidos PRA.xlsx"
# Copiar el archivo


Pegar y modificar venta sug

In [ ]:
import pandas as pd
import numpy as np
import warnings
import os
import shutil
from datetime import datetime, timedelta
import win32com.client as win32
import time


#Definir Semana
ANO_ARCHIVO = 2024
N_Mes = "Jun"
Mes = "6"
Mescon0 = fr"0{Mes}"

# Obtener la ruta de los de los Archivos de Ventas Diarias
Ruta_BI = "C:\\Users\\PabloMenendez\\Franquicias Americanas SA de CV\\PRA-BI - PRA-BI\\"
carpeta_cwsv = fr"{Ruta_BI}5. CW CAM\\1. CW SV\\1. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_phgt = fr"{Ruta_BI}3. PH GT\\1. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_phsv = fr"{Ruta_BI}2. PH SV\\1. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_wdsv = fr"{Ruta_BI}7. WD SV\\1. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_stb = fr"{Ruta_BI}6. STB CAM\\0. STB CAM\\1. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_kfcsv = fr"{Ruta_BI}4. KFC CAM\\1. KFC SV\\0. Ventas Diarias\\{ANO_ARCHIVO}\\"
carpeta_kfcgt = fr"{Ruta_BI}4. KFC CAM\\2. KFC GT\\0. Ventas Diarias\\{ANO_ARCHIVO}\\"


# Crear un diccionario de rutas con las rutas relativas a la carpeta "Ventas Diarias"
Rutas = {fr"{carpeta_cwsv}{Mescon0}. Ventas Diarias CW SV {N_Mes} {ANO_ARCHIVO}.xlsx": ["Pivot CW SV.xlsx"],
         fr"{carpeta_phgt}{Mes}. Ventas Diarias PH GT {N_Mes} {ANO_ARCHIVO}.xlsx": ["Pivot PH GT.xlsx"],
         fr"{carpeta_phsv}{Mescon0}. Ventas Diarias PH SV {N_Mes} {ANO_ARCHIVO}.xlsx": ["Pivot PH SV.xlsx"],
         fr"{carpeta_wdsv}{Mescon0}. Ventas Diarias WD SV {N_Mes} {ANO_ARCHIVO}.xlsx": ["Pivot WD SV.xlsx"],
         fr"{carpeta_stb}{Mescon0}. Ventas Diarias STB CAM {N_Mes} {ANO_ARCHIVO}.xlsx": ["Pivot STB CAM.xlsx"],
         fr"{carpeta_kfcsv}{Mescon0}. Ventas Diarias KFC SV {N_Mes} {ANO_ARCHIVO}.xlsx": ["Pivot KFC SV.xlsx"],
         fr"{carpeta_kfcgt}{Mes}. Ventas Diarias KFC GT {N_Mes} {ANO_ARCHIVO}.xlsx": ["Pivot KFC GT.xlsx"]
}

lista_marcas = []


for marca, atributos in Rutas.items():

    Ruta_Destino = fr"C:/Users/PabloMenendez/Reportería/Ventas Total Premium/Pivots Ventas Diarias/{atributos[0]}"
    # Copiar el archivo
    shutil.copy(marca,Ruta_Destino)
    #Crear lista de Archivos
    lista_marcas.append(atributos[0])

import win32com.client as win32
excel = win32.Dispatch("Excel.Application")
excel.DisplayAlerts = False
excel.Visible = True



for archivo in lista_marcas:
    wb = excel.Workbooks.Open(fr"C:/Users/PabloMenendez/Reportería/Ventas Total Premium/Pivots Ventas Diarias/{archivo}", UpdateLinks=0)

    ws = wb.Worksheets("Base P.R.A.")
    ws.Visible = True
    wb.Worksheets("Base P.R.A.").Activate()
    pivot_table = ws.PivotTables(1)

    for campo in pivot_table.PivotFields():
        if campo.Name ==  "MES" or campo.Name ==  "ANIO" or campo.Name ==  "FECHA_DOCUMENTO":
            campo.ClearAllFilters()

    time.sleep(5)
    wb.Save()
    wb.Close()
excel.Quit()

Deck comercial

In [ ]:
## Importar Equivalencias ##

Anexos = "C:\\Users\\PabloMenendez\\Reportería\\Deck Delivery Premium 2024\\Anexos\\"
Anexo_MIX_Periodo = pd.read_excel(fr"{Anexos}MIX Periodos.xlsx")
Anexo_WPSA_Cambio = pd.read_excel(fr"{Anexos}WPSA Cambio.xlsx")



## Definir Rutas de Archivo ##
Ruta_BI = "C:\\Users\\PabloMenendez\\Franquicias Americanas SA de CV\\PRA-BI - PRA-BI\\"

carpeta_cwsv = fr"{Ruta_BI}5. CW CAM\\1. CW SV\\7. Deck Comercial\\2024\\"
carpeta_phgt = fr"{Ruta_BI}3. PH GT\\5. Mix Management Tool\\1. Semanal\\"
carpeta_phsv = fr"{Ruta_BI}2. PH SV\\7. Deck Comercial\\2024\\"
carpeta_wdsv = fr"{Ruta_BI}7. WD SV\\7. Deck Comercial\\Nuevo Deck Comercial\\"
carpeta_stbsv = fr"{Ruta_BI}6. STB CAM\\1. STB SV\\6. Deck Comercial\\Nuevo Deck Comercial\\"
carpeta_stbgt = fr"{Ruta_BI}6. STB CAM\\2. STB GT\\6. Deck Comercial\\Nuevo Deck Comercial\\"
carpeta_stbcr = fr"{Ruta_BI}6. STB CAM\\3. STB CR\\6. Deck Comercial\\Nuevo Deck Comercial\\"
carpeta_kfcsv = fr"{Ruta_BI}4. KFC CAM\\1. KFC SV\\6. Deck Comercial\\2024\\"
carpeta_kfcgt = fr"{Ruta_BI}4. KFC CAM\\2. KFC GT\\4. Mix Management Tool\\1. Semanal\\"
Rutas = {
    fr"{carpeta_cwsv}2024 CW SV Deck Comercial Tabla Productos, Graf Capas.xlsx": ["Base MMT P.R.A.", "CW SV",0]
    ,fr"{carpeta_phgt}Mix Management Tool Sem PH GT {ANO_ARCHIVO}.xlsm": ["Base MMT P.R.A.","PH GT",4]
    ,fr"{carpeta_phsv}2024 PH SV Deck Comercial Tabla Productos, Graf Capas.xlsx": ["Base MMT P.R.A.","PH SV",0]
    ,fr"{carpeta_wdsv}Deck Comercial WD SV Nueva Versión.xlsx": ["Base MMT P.R.A.","WD SV",0]
    ,fr"{carpeta_stbgt}Deck Comercial STB GT Nueva Versión.xlsx": ["Base MMT P.R.A.","STB GT",0]
    ,fr"{carpeta_stbsv}Deck Comercial STB SV Nueva Versión.xlsx": ["Base MMT P.R.A.","STB SV",0]
    ,fr"{carpeta_stbcr}Deck Comercial STB CR Nueva Versión.xlsx": ["Base MMT P.R.A.","STB CR",0]
    ,fr"{carpeta_kfcgt}Mix Management Tool - KFC GT Sem {ANO_ARCHIVO}.xlsm": ["Base MMT P.R.A.","KFC GT",3]
    ,fr"{carpeta_kfcsv}2024 KFC SV Deck Comercial Tabla Productos, Graf Capas.xlsx": ["Base MMT P.R.A.","KFC SV",0]
    }


## Definir el df consolidado y Los Canales a filtrar
profit = pd.DataFrame()
Canales = ["DOMICILIO","ON LINE"]

## Obtener información de Profit ##
for marca, pivot in Rutas.items():

    #Obtener información de cada marca
    marca = pd.read_excel(marca, sheet_name=pivot[0],skiprows=pivot[2])


    #Eliminar fila 'Unnamed: 0' para archivos que posean la primera columna sin datos
    if 'Unnamed: 0' in marca.columns:
        marca.drop('Unnamed: 0', axis=1, inplace=True)

    #Cambiar el nombre de las columnas
    Nuevas_columnas = ['ANIO PRA', 'SEMANA','CANAL', 'TIENDA','CATEGORIA','VT','COSTE','PROFIT','CANT']
    marca.columns = Nuevas_columnas

    #Agregar columna Marca
    marca.insert(0,"MARCA",pivot[1])

    #Agregar Equivalencia de Periodo
    marca = marca.merge(Anexo_MIX_Periodo, on=['ANIO PRA','SEMANA'], how='left')

    #Filtrar canal a unicamente domicilio
    marca = marca[marca["CANAL"].isin(Canales)]

    #Realizar un groupby para quedarse con las columnas deseadas
    marca = marca.groupby(["MARCA",'ANIO PRA', 'PERIODO', 'SEMANA',
                        'TIENDA']
                        ).agg({'VT': 'sum', 'COSTE': 'sum', 'PROFIT': 'sum', 'CANT': 'sum'}).reset_index()

    #Obtner la ultima semana de actualizacion
    anio = marca[marca["ANIO PRA"]==ANO_ARCHIVO]
    semana_max = anio["SEMANA"].max()
    print(fr"{pivot[1]} Semana {semana_max}")


    #Obtener porcentaje de Profit
    marca['%Profit']= marca['PROFIT']/marca['VT']

    #Limpieza de la columna Tienda
    marca['TIENDA'] = marca['TIENDA'].str.upper()
    marca['TIENDA'] = marca['TIENDA'].str.strip()
    marca['ANIO PRA'] = marca['ANIO PRA'].astype(str)


    #Filtrar df final y agregarlo al consolidado
    marca = marca[['MARCA', 'ANIO PRA', 'PERIODO', 'SEMANA', 'TIENDA', '%Profit']]
    profit= pd.concat([profit, marca], ignore_index=True)


#Descargar Archivo de Profit
Archivo_profit = fr"{Anexos}Profit Domicilio.xlsx"
profit.to_excel(Archivo_profit, index=False)

print("Finalizado")

Enviar correos

In [ ]:
def enviar_correo(x,y,ARCHIVO,CORREOS):
    outlook = win32.Dispatch("Outlook.Application")
    correo = outlook.CreateItem(0)  # 0 significa un correo nuevo

    # Configurar los campos del correo
    correo.Subject = fr"Venta por Cuadrante {x} [Sem {SEMANA}]"
    correo.Body = (f"Buen día,\nComparto el archivo de Venta por Cuadrante de {y} actualizado hasta la semana {SEMANA}.\n"
                    "Cualquier consulta estoy a la orden.\n\n"
                    "Saludos,\n"
                    "Pablo Menéndez")
    correo.To = CORREOS  # Dirección de correo electrónico del destinatario
    correo.CC = "zayra.alvarenga@premium.sv; jesus.argueta@premium.sv; cristian.cornejo@premium.sv; edson.mejia@premium.sv; julio.landaverde@premium.sv"

    archivo_adjunto = f"C:/Users/PabloMenendez/Reportería/Venta Cuadrantes/{ARCHIVO}"  # Ruta del archivo a adjuntar
    correo.Attachments.Add(archivo_adjunto)
    # Enviar el correo
    correo.Send()

    print("Correo enviado correctamente")

for marca,valor in Datos.items():
    enviar_correo(valor[0],valor[1],valor[2],valor[3])

Base V sug WD

In [ ]:
with wendys as (select * from
(select todo.fecha ,todo.marca,to_char(todo.fecha, 'dd') as dia,
       to_char(todo.fecha,'yyyy') as anio, initcap(to_char(todo.fecha, 'MONTH')) as mes,
       todo.operador, todo.ordenes,
       todo.cantidad_frosty16, todo.venta_frosty16, round(todo.cantidad_frosty16/todo.ordenes, 2) as vsug_frosty16,
       --agrandados
       otro.cantidad_agrandado035, otro.venta_agrandado035, round(otro.cantidad_agrandado035/todo.ordenes, 2) as vsug_agrandado035,
       otro.cantidad_agrandado070, otro.venta_agrandado070, round(otro.cantidad_agrandado070/todo.ordenes, 2) as vsug_agrandado070,
       otro.cantidad_agrandado105, otro.venta_agrandado105, round(otro.cantidad_agrandado105/todo.ordenes, 2) as vsug_agrandado105,
       otro.cantidad_agrandado025, otro.venta_agrandado025, round(otro.cantidad_agrandado025/todo.ordenes, 2) as vsug_agrandado025,
       otro.cantidad_agrandado060, otro.venta_agrandado060, round(otro.cantidad_agrandado060/todo.ordenes, 2) as vsug_agrandado060,
       otro.cantidad_agrandado095, otro.venta_agrandado095, round(otro.cantidad_agrandado095/todo.ordenes, 2) as vsug_agrandado095,
       otro.cantidad_spicyfries, otro.venta_spicyfries, round(otro.cantidad_spicyfries/todo.ordenes, 2) as vsug_spicyfries,


       todo.cantidad_queso059, todo.venta_queso059, round(todo.cantidad_queso059/todo.ordenes, 2) as vsug_queso059,
       todo.cantidad_qralla105, todo.venta_qralla105, round(todo.cantidad_qralla105/todo.ordenes, 2) as vsug_qralla105,
       --queso rallado chili
       otro.cantidad_qrallachili, otro.venta_qrallachili, round(otro.cantidad_qrallachili/todo.ordenes, 2) as vsug_qrallachili,

       todo.cantidad_salque045, todo.venta_salque045, round(todo.cantidad_salque045/todo.ordenes, 2) as vsug_salque045,
       todo.cantidad_pie105, todo.venta_pie105, round(todo.cantidad_pie105/todo.ordenes, 2) as vsug_pie105,
       todo.cantidad_pie175, todo.venta_pie175, round(todo.cantidad_pie175/todo.ordenes, 2) as vsug_pie175,
       todo.cantidad_pie265, todo.venta_pie265, round(todo.cantidad_pie265/todo.ordenes, 2) as vsug_pie265,
       todo.cantidad_yogurt, todo.venta_yogurt, round(todo.cantidad_yogurt/todo.ordenes, 2) as vsug_yogurt,
       otro.cantidad_frostyccino, otro.venta_frostyccino, round(otro.cantidad_frostyccino/todo.ordenes, 2) as vsug_frostyccino,
       otro.cantidad_limoensa16, otro.venta_limoensa16, round(otro.cantidad_limoensa16/todo.ordenes, 2) as vsug_limoensa16,
       otro.cantidad_limoensa20, otro.venta_limoensa20, round(otro.cantidad_limoensa20/todo.ordenes, 2) as vsug_limoensa20,
       otro.cantidad_limocombo16, otro.venta_limocombo16, round(otro.cantidad_limocombo16/todo.ordenes, 2) as vsug_limocombo16,
       otro.cantidad_limocombo20, otro.venta_limocombo20, round(otro.cantidad_limocombo20/todo.ordenes, 2) as vsug_limocombo20,
       todo.venta

from (

select ord.fecha,orc.codcia as marca, nvl(orc.codusr, '-') as operador, count(distinct(orc.numord_cia)) as ordenes, round(sum(dor.total_linea)/1.13, 2) as venta,
       sum(case when dor.codmnu in ('FRCOOK','FRCHO','FRMIX','FRVAIN') then dor.cantidad else 0
       end) as cantidad_frosty16,
       round(sum(case when dor.codmnu in ('FRCOOK','FRCHO','FRMIX','FRVAIN') then dor.total_linea else 0
       end)/1.13, 2) as venta_frosty16,

       sum(case when dor.codmnu in ('QUESO') then dor.cantidad else 0
       end) as cantidad_queso059,
       round(sum(case when dor.codmnu in ('QUESO') then dor.total_linea else 0
       end)/1.13, 2) as venta_queso059,

       sum(case when dor.codmnu in ('QRALLA') then dor.cantidad else 0
       end) as cantidad_qralla105,
       round(sum(case when dor.codmnu in ('QRALLA') then dor.total_linea else 0
       end)/1.13, 2) as venta_qralla105,

       sum(case when dor.codmnu in ('SALQUE') then dor.cantidad else 0
       end) as cantidad_salque045,
       round(sum(case when dor.codmnu in ('SALQUE') then dor.total_linea else 0
       end)/1.13, 2) as venta_salque045,

       sum(case when dor.codmnu in ('PASTEL') then dor.cantidad else 0
       end) as cantidad_pie105,
       round(sum(case when dor.codmnu in ('PASTEL') then dor.total_linea else 0
       end)/1.13, 2) as venta_pie105,

       sum(case when dor.codmnu in ('2PIE') then dor.cantidad else 0
       end) as cantidad_pie175,
       round(sum(case when dor.codmnu in ('2PIE') then dor.total_linea else 0
       end)/1.13, 2) as venta_pie175,

       sum(case when dor.codmnu in ('3PIE') then dor.cantidad else 0
       end) as cantidad_pie265,
       round(sum(case when dor.codmnu in ('3PIE') then dor.total_linea else 0
       end)/1.13, 2) as venta_pie265,

       sum(case when
       (dor.codmnu in ('YOG') --Venta de yogurt
            and exists (
                        select 1 from ccrall.ccr_det_orden dor2
                        where orc.numord_cia = dor2.numord_cia
                        and dor2.codmnu in ('SPCESARC', 'APPLEC', 'STWESTC', 'WRAPSB1', 'WPBPC1', 'WRAPSW',
                                            'KPAN1', 'MUFFIN1', 'PANC', 'TIPICO', 'DBTIPICO', 'CDTIPICO', 'CTMDH1', 'PKMDH', 'CDERRE',
                                            '3BKFAST', '4BKFAST', 'BIGGIEM', 'TRPCROS', 'FAMCROS', 'DUOBKML', 'DUOBKBN', 'KMUF1', '4PKTIPICO', 'BFAMPACK'
                                            )))
       then dor.cantidad else 0 end) as cantidad_yogurt,

       round(sum(case when
       (dor.codmnu in ('YOG') --Venta de yogurt
            and exists (
                        select 1 from ccrall.ccr_det_orden dor2
                        where orc.numord_cia = dor2.numord_cia
                        and dor2.codmnu in ('SPCESARC', 'APPLEC', 'STWESTC', 'WRAPSB1', 'WPBPC1', 'WRAPSW',
                                            'KPAN1', 'MUFFIN1', 'PANC', 'TIPICO', 'DBTIPICO', 'CDTIPICO', 'CTMDH1', 'PKMDH', 'CDERRE',
                                            '3BKFAST', '4BKFAST', 'BIGGIEM', 'TRPCROS', 'FAMCROS', 'DUOBKML', 'DUOBKBN', 'KMUF1', '4PKTIPICO', 'BFAMPACK'
                                            )))
       then dor.total_linea else 0 end)/1.13, 2) as venta_yogurt

--dor.codmnu in ('3PIE')
from ccrall.ccr_ordenes ord
join ccrall.ccr_ordenes_cia orc on ord.numord = orc.numord
join ccrall.ccr_det_orden dor on orc.numord_cia = dor.numord_cia and orc.codcia = dor.codcia

where ord.fecha between to_date(:F_INI,'yyyy-mm-dd') and  to_date(:F_FIN,'yyyy-mm-dd')
  and orc.estado in ('ENV','ENF')
  and orc.origen_pedido in ('CCR', 'WAP')

  --Exclusion de tiendas segun compañia
  and orc.codcia='WDYSAL'
  and orc.codtda not in ('100', '001')
  --and dor.codmnu in ('FRCOOK','FRCHO','FRMIX','FRVAIN')

group by ord.fecha,orc.codcia, nvl(orc.codusr, '-')

) todo

left join

(select ord.fecha, nvl(orc.codusr, '-') as operador,
       sum(case when
       ( (cor.coddom in ('2') and cor.codopm in ('2'))
       or (cor.coddom in ('COMP2') and cor.codopm in ('CMBCOM'))
       or (cor.coddom in ('COMPEW02') and cor.codopm in ('COMPEW'))
       ) then dor.cantidad else 0 end) as cantidad_agrandado035,

       round(sum(case when
       ( (cor.coddom in ('2') and cor.codopm in ('2'))
       or (cor.coddom in ('COMP2') and cor.codopm in ('CMBCOM'))
       or (cor.coddom in ('COMPEW02') and cor.codopm in ('COMPEW'))
       ) then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_agrandado035,

       sum(case when
       ( (cor.coddom in('3') and cor.codopm in('2'))
       or (cor.coddom in('COMP3') and cor.codopm in('CMBCOM'))
       or (cor.coddom in('COMPEW03') and cor.codopm in('COMPEW'))
       ) then dor.cantidad else 0 end) as cantidad_agrandado070,

       round(sum(case when
       ( (cor.coddom in('3') and cor.codopm in('2'))
       or (cor.coddom in('COMP3') and cor.codopm in('CMBCOM'))
       or (cor.coddom in('COMPEW03') and cor.codopm in('COMPEW'))
       ) then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_agrandado070,

       sum(case when
       (cor.coddom in('COMPEW04') and cor.codopm in('COMPEW') )
       then dor.cantidad else 0 end) as cantidad_agrandado105,

       round(sum(case when
       (cor.coddom in('COMPEW04') and cor.codopm in('COMPEW') )
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_agrandado105,

       sum(case when
       (cor.coddom in('13') and cor.codopm in('CMBCOM'))
       then dor.cantidad else 0 end) as cantidad_agrandado025,

       round(sum(case when
       (cor.coddom in('13') and cor.codopm in('CMBCOM'))
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_agrandado025,

       sum(case when
       (cor.coddom in('14') and cor.codopm in('CMBCOM'))
       then dor.cantidad else 0 end) as cantidad_agrandado060,

       round(sum(case when
       (cor.coddom in('14') and cor.codopm in('CMBCOM'))
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_agrandado060,

       sum(case when
       (cor.coddom in('15') and cor.codopm in('CMBCOM'))
       then dor.cantidad else 0 end) as cantidad_agrandado095,

       round(sum(case when
       (cor.coddom in('15') and cor.codopm in('CMBCOM'))
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_agrandado095,

       sum(case when
       (cor.codopm in ('CHILIGDE') and cor.coddom in ('CH03'))
       then dor.cantidad else 0 end) as cantidad_qrallachili,

       round(sum(case when
       (cor.codopm in ('CHILIGDE') and cor.coddom in ('CH03'))
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_qrallachili,

       sum(case when
       (dor.codmnu in ('BKFBACO', 'BKFBASW', 'BKFCHI', 'BKFSWIS', 'BKFDBSAL', 'BKFMELT')
       and cor.codopm in ('JUGOC') and cor.coddom in ('13', '14'))
       then dor.cantidad else 0 end) as cantidad_frostyccino,

       round(sum(case when
       (dor.codmnu in ('BKFBACO', 'BKFBASW', 'BKFCHI', 'BKFSWIS', 'BKFDBSAL', 'BKFMELT')
       and cor.codopm in ('JUGOC') and cor.coddom in ('13', '14'))
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_frostyccino,

       sum(case when
       (dor.codmnu in ('CLASSIC', 'DOUBLE', 'TRIPLE', 'POP', 'PEMPAN', 'SPICY',
                        'CSPGCO', 'CSPGUA', 'BBC', 'BBC2', 'BBC3', 'BACONATOR',
                        'BNATOR', 'MELT', 'BBLMELT', 'CBBP','DOBDA', 'CBBPC',
                        'CHILBC', 'DOBCB', 'WRAPSB1', 'WPBPC1', 'WRAPSW', 'DOBPA', 'DBLMELT')  --Venta de spicy fries de 1.10
                and cor.codopm in ('CMBCOM') and cor.coddom in ('8'))
       then dor.cantidad else 0 end) as cantidad_spicyfries,

       round(sum(case when
       (dor.codmnu in ('CLASSIC', 'DOUBLE', 'TRIPLE', 'POP', 'PEMPAN', 'SPICY',
                        'CSPGCO', 'CSPGUA', 'BBC', 'BBC2', 'BBC3', 'BACONATOR',
                        'BNATOR', 'MELT', 'BBLMELT', 'CBBP','DOBDA', 'CBBPC',
                        'CHILBC', 'DOBCB', 'WRAPSB1', 'WPBPC1', 'WRAPSW', 'DOBPA', 'DBLMELT')  --Venta de spicy fries de 1.10
                and cor.codopm in ('CMBCOM') and cor.coddom in ('8'))
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_spicyfries,

       sum(case when
       (dor.codmnu in ('SPCESARC', 'APPLEC', 'STWESTC')  --cantidad de limonada 16 oz ensalada
        and cor.codopm in ('SODAS') and cor.coddom in ('22', '15', '18'))
       then dor.cantidad else 0 end) as cantidad_limoensa16,

       round(sum(case when
       (dor.codmnu in ('SPCESARC', 'APPLEC', 'STWESTC')  --Venta de limonada 16 oz ensalada
        and cor.codopm in ('SODAS') and cor.coddom in ('22', '15', '18'))
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_limoensa16,

       sum(case when
       (dor.codmnu in ('SPCESARC', 'APPLEC', 'STWESTC')  --cantidad de limonada 20 oz ensalada
        and cor.codopm in ('SODAS') and cor.coddom in ('23', '16', '19'))
       then dor.cantidad else 0 end) as cantidad_limoensa20,

       round(sum(case when
       (dor.codmnu in ('SPCESARC', 'APPLEC', 'STWESTC')  --Venta de limonada 20 oz ensalada
        and cor.codopm in ('SODAS') and cor.coddom in ('23', '16', '19'))
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_limoensa20,

       sum(case when
       (dor.codmnu in ('CLASSIC', 'DOUBLE', 'TRIPLE', 'POP', 'PEMPAN', 'SPICY',
                        'CSPGCO', 'CSPGUA', 'BBC', 'BBC2', 'BBC3', 'BACONATOR',
                        'BNATOR', 'MELT', 'BBLMELT', 'CBBP','DOBDA', 'CBBPC',
                        'CHILBC', 'DOBCB', 'WRAPSB1', 'WPBPC1', 'WRAPSW', 'DOBPA', 'DBLMELT')  --Cantidad de limonada 16 oz combos
        and cor.codopm in ('SODAS') and cor.coddom in ('22', '15', '18'))
       then dor.cantidad else 0 end) as cantidad_limocombo16,

       round(sum(case when
       (dor.codmnu in ('CLASSIC', 'DOUBLE', 'TRIPLE', 'POP', 'PEMPAN', 'SPICY',
                        'CSPGCO', 'CSPGUA', 'BBC', 'BBC2', 'BBC3', 'BACONATOR',
                        'BNATOR', 'MELT', 'BBLMELT', 'CBBP','DOBDA', 'CBBPC',
                        'CHILBC', 'DOBCB', 'WRAPSB1', 'WPBPC1', 'WRAPSW', 'DOBPA', 'DBLMELT')  --Venta de limonada 16 oz combos
        and cor.codopm in ('SODAS') and cor.coddom in ('22', '15', '18'))
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_limocombo16,

       sum(case when
       (dor.codmnu in ('CLASSIC', 'DOUBLE', 'TRIPLE', 'POP', 'PEMPAN', 'SPICY',
                        'CSPGCO', 'CSPGUA', 'BBC', 'BBC2', 'BBC3', 'BACONATOR',
                        'BNATOR', 'MELT', 'BBLMELT', 'CBBP','DOBDA', 'CBBPC',
                        'CHILBC', 'DOBCB', 'WRAPSB1', 'WPBPC1', 'WRAPSW', 'DOBPA', 'DBLMELT')  --Cantidad de limonada 20 oz combos
        and cor.codopm in ('SODAS') and cor.coddom in ('23', '16', '19'))
       then dor.cantidad else 0 end) as cantidad_limocombo20,

       round(sum(case when
       (dor.codmnu in ('CLASSIC', 'DOUBLE', 'TRIPLE', 'POP', 'PEMPAN', 'SPICY',
                        'CSPGCO', 'CSPGUA', 'BBC', 'BBC2', 'BBC3', 'BACONATOR',
                        'BNATOR', 'MELT', 'BBLMELT', 'CBBP','DOBDA', 'CBBPC',
                        'CHILBC', 'DOBCB', 'WRAPSB1', 'WPBPC1', 'WRAPSW', 'DOBPA', 'DBLMELT')  --Venta de limonada 20 oz combos
        and cor.codopm in ('SODAS') and cor.coddom in ('23', '16', '19'))
       then (dor.cantidad * dom.costo_extra) else 0 end)/1.13, 2) as venta_limocombo20

--dor.codmnu in ('3PIE')
from ccrall.ccr_ordenes ord
join ccrall.ccr_ordenes_cia orc on ord.numord = orc.numord
join ccrall.ccr_det_orden dor on orc.numord_cia = dor.numord_cia and orc.codcia = dor.codcia
join ccrall.ccr_comp_orden cor on dor.iddor = cor.iddor
join ccrall.ccr_det_opciones_menu dom on cor.codopm = dom.codopm and cor.coddom = dom.coddom

where ord.fecha between to_date(:F_INI,'yyyy-mm-dd') and  to_date(:F_FIN,'yyyy-mm-dd')
  and orc.estado in ('ENV','ENF')
  and orc.origen_pedido in ('CCR', 'WAP')

  --Exclusion de tiendas segun compañia
  and orc.codcia='WDYSAL'
  and orc.codtda not in ('100', '001')


group by ord.fecha, nvl(orc.codusr, '-')

) otro on (todo.fecha = otro.fecha and todo.operador = otro.operador)
))
select marca, fecha, operador, Programa,cantidad,Venta, comision ,Universo
, case when Programa in ('Frosty 16 oz') then 'Frosty'
when Programa in ('Agrandado Papa $0.35','Agrandado Papa $0.70','Agrandado Papa $1.05','Agrandado Red Fries $0.25','Agrandado Red Fries $0.60','Agrandado Red Fries $0.95','Spicy Fries') then 'Agrandado Papa'
when Programa in ('Queso Lasca','Queso Rallado','Queso_Ralla_Chili','Queso Derretido') then 'Queso'
when Programa in ('Pie $1.65','Pie $1.75','Pie $2.65') then 'Apple Pie'
when Programa in ('Yogurt') then 'Yogurt'
when Programa in ('Frostyccino') then 'Frostyccino'
when Programa in ('Limonada Ensaladas 16 oz','Limonada Ensaladas 20 oz','Limonada Combo 16 oz','Limonada Combo 20 oz') then 'Limonada'
else 'Ordenes' end as Categoría
from(
--Ordenes
select distinct fecha,marca,operador,'Ordenes' as Programa,ordenes as Cantidad,venta as Venta,0 as Comision,0 as Universo from wendys
--Frosty16
UNION ALL select distinct fecha,marca,operador,'Frosty 16 oz' as Programa,cantidad_frosty16 as Cantidad,venta_frosty16 as Venta,vsug_frosty16 as Comision,ordenes as Universo from wendys
--Agrandados
UNION ALL select distinct fecha, marca, operador, 'Agrandado Papa $0.35' as Programa, cantidad_agrandado035 as Cantidad, venta_agrandado035 as Venta, vsug_agrandado035 as Comision,ordenes as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Agrandado Papa $0.70' as Programa, cantidad_agrandado070 as Cantidad, venta_agrandado070 as Venta, vsug_agrandado070 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Agrandado Papa $1.05' as Programa, cantidad_agrandado105 as Cantidad, venta_agrandado105 as Venta, vsug_agrandado105 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Agrandado Red Fries $0.25' as Programa, cantidad_agrandado025 as Cantidad, venta_agrandado025 as Venta, vsug_agrandado025 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Agrandado Red Fries $0.60' as Programa, cantidad_agrandado060 as Cantidad, venta_agrandado060 as Venta, vsug_agrandado060 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Agrandado Red Fries $0.95' as Programa, cantidad_agrandado095 as Cantidad, venta_agrandado095 as Venta, vsug_agrandado095 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Spicy Fries' as Programa, cantidad_spicyfries as Cantidad, venta_spicyfries as Venta, vsug_spicyfries as Comision,0 as Universo from wendys
-- Queso y otros
UNION ALL select distinct fecha, marca, operador, 'Queso Lasca' as Programa, cantidad_queso059 as Cantidad, venta_queso059 as Venta, vsug_queso059 as Comision,ordenes as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Queso Rallado' as Programa, cantidad_qralla105 as Cantidad, venta_qralla105 as Venta, vsug_qralla105 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Queso_Ralla_Chili' as Programa, cantidad_qrallachili as Cantidad, venta_qrallachili as Venta, vsug_qrallachili as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Queso Derretido' as Programa, cantidad_salque045 as Cantidad, venta_salque045 as Venta, vsug_salque045 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Pie $1.65' as Programa, cantidad_pie105 as Cantidad, venta_pie105 as Venta, vsug_pie105 as Comision,ordenes as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Pie $1.75' as Programa, cantidad_pie175 as Cantidad, venta_pie175 as Venta, vsug_pie175 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Pie $2.65' as Programa, cantidad_pie265 as Cantidad, venta_pie265 as Venta, vsug_pie265 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Yogurt' as Programa, cantidad_yogurt as Cantidad, venta_yogurt as Venta, vsug_yogurt as Comision,ordenes as Universo from wendys
-- Otros productos
UNION ALL select distinct fecha, marca, operador, 'Frostyccino' as Programa, cantidad_frostyccino as Cantidad, venta_frostyccino as Venta, vsug_frostyccino as Comision,ordenes as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Limonada Ensaladas 16 oz' as Programa, cantidad_limoensa16 as Cantidad, venta_limoensa16 as Venta, vsug_limoensa16 as Comision,ordenes as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Limonada Ensaladas 20 oz' as Programa, cantidad_limoensa20 as Cantidad, venta_limoensa20 as Venta, vsug_limoensa20 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Limonada Combo 16 oz' as Programa, cantidad_limocombo16 as Cantidad, venta_limocombo16 as Venta, vsug_limocombo16 as Comision,0 as Universo from wendys
UNION ALL select distinct fecha, marca, operador, 'Limonada Combo 20 oz' as Programa, cantidad_limocombo20 as Cantidad, venta_limocombo20 as Venta, vsug_limocombo20 as Comision,0 as Universo from wendys
) resultado
ORDER BY fecha,
operador,
CASE Programa WHEN 'Ordenes' THEN 1 WHEN 'Frosty16' THEN 2 WHEN 'Agrandado_0.35' THEN 3 WHEN 'Agrandado_0.70' THEN 4 WHEN 'Agrandado_1.05' THEN 5 WHEN 'Agrandado_0.25' THEN 6
WHEN 'Agrandado_0.60' THEN 7 WHEN 'Agrandado_0.95' THEN 8 WHEN 'Spicy Fries' THEN 9 WHEN 'Queso_0.59' THEN 10 WHEN 'Queso_Ralla_1.05' THEN 11 WHEN 'Queso_Ralla_Chili' THEN 12
WHEN 'Salque_0.45' THEN 13 WHEN 'Pie_1.05' THEN 14 WHEN 'Pie_1.75' THEN 15 WHEN 'Pie_2.65' THEN 16 WHEN 'Yogurt' THEN 17 WHEN 'Frostyccino' THEN 18 WHEN 'Limoensa_16' THEN 19
WHEN 'Limoensa_20' THEN 20 WHEN 'Limocombo_16' THEN 21 WHEN 'Limocombo_20' THEN 22 ELSE 23 END

Venta cuadrantes

In [ ]:
ruta = 'C:/Users/PabloMenendez/Reportería/Venta Cuadrantes/Consultas/'
ruta_anexos = 'C:/Users/PabloMenendez/Reportería/Venta Cuadrantes/Anexos/'
ruta_bases = 'C:/Users/PabloMenendez/Reportería/Venta Cuadrantes/BASES/'

ruta_gerente = fr"{ruta_anexos}Gerente.xlsx"
ruta_sem = fr"{ruta_anexos}Semana.xlsx"

Gerente = pd.read_excel(ruta_gerente)
Semanas = pd.read_excel(ruta_sem)

Rutas = { fr"{ruta}Cuadrantes CW SV {ANO}.sql": ["CW SV"],
          fr"{ruta}Cuadrantes PH GT {ANO}.sql": ["PH GT"],
          fr"{ruta}Cuadrantes PH SV {ANO}.sql": ["PH SV"],
          fr"{ruta}Cuadrantes WD SV {ANO}.sql": ["WD SV"],
          fr"{ruta}Cuadrantes KFC SV {ANO}.sql": ["KFC SV"],
          fr"{ruta}Cuadrantes KFC GT {ANO}.sql": ["KFC GT"]
        }
for archivo, nombre in Rutas.items():
    BASE = archivo
    with open(BASE, 'r') as sql_script:
     sentencia = sql_script.read()

    script_final = sentencia.replace(':FECHA_INICIO', f_ini)
    script_final = script_final.replace(':FECHA_FIN', f_fin)

    #Crear conexión a la base de datos
    p_dsn = ora.makedsn("172.16.100.207", 1521, sid = "ccrdb")

    con = ora.connect(user = "BI_ANALISTA1", password = "PREMIUM2022#"
                      , dsn = p_dsn, encoding = "UTF-8")
    cursor = con.cursor()
    rs = cursor.execute(script_final)

    #Concatenar filas
    Base = []
    for fila in rs:
        Base.append(list(fila))
    con.close()
    print(f'Consulta {nombre[0]} Importada')
    #tranbsformaciones
    df = pd.DataFrame(Base)
    df.columns = ['ANIO','ANIO_PRA','MES', 'SEMANA'
                  , 'CODCIA', 'IDTDA', 'TIENDA','CUADRANTE','CODCOL'
                 , 'COLONIA','ORIGEN','Tx','VT']
    df.insert(5,"MARCA",nombre[0])
    df.drop('CODCIA', axis=1, inplace=True)

    MES = df['MES'].max()
    ANIO = df['ANIO'].max()

    df['ANIO_PRA'] = df['ANIO_PRA'].astype('int64')
    df['SEMANA'] = df['SEMANA'].astype('int64')

    df = df.merge(Gerente, on=["MARCA",'IDTDA'], how='left')
    df = df.merge(Semanas, on=['ANIO_PRA','SEMANA'], how='left')
    df = df.fillna('-')
    SEMANA = df['SEMANA'].max()

    Ruta_final = fr"{ruta_bases}{nombre[0]}/Cuadrantes {nombre[0]} {MES}-{ANO}.xlsx"
    df.to_excel(Ruta_final, index